# Job Board Scraping Lab

In this lab you will first see a minimal but fully functional code snippet to scrape the LinkedIn Job Search webpage. You will then work on top of the example code and complete several chanllenges.

### Some Resources 

- [Requests library](http://docs.python-requests.org/en/master/#the-user-guide) documentation 
- [Beautiful Soup Doc](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Urllib](https://docs.python.org/3/library/urllib.html#module-urllib)
- [re lib](https://docs.python.org/3/library/re.html)
- [Scrapy](https://scrapy.org/)
- [List of HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes)
- [HTML basics](http://www.simplehtmlguide.com/cheatsheet.php)
- [CSS basics](https://www.cssbasics.com/#page_start)

In [3]:
# Import the required libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests

"""
This function searches job posts from LinkedIn and converts the results into a dataframe.
"""
def scrape_linkedin_job_search(keywords):
    
    # Define the base url to be scraped.
    # All uppercase variable name signifies this is a constant and its value should never unchange
    BASE_URL = 'https://www.linkedin.com/jobs/search/?'
    
    # Assemble the full url with parameters
    scrape_url = ''.join([BASE_URL, 'keywords=', keywords])

    # Create a request to get the data from the server 
    page = requests.get(scrape_url)
    soup = BeautifulSoup(page.text, 'html.parser')


    # Create an empty dataframe with the columns consisting of the information you want to capture
    columns = ['Title', 'Company', 'Location']
    data = pd.DataFrame(columns=columns)

    # Retrieve HTML code from the webpage. Parse the HTML into a list of "cards".
    # Then in each job card, extract the job title, company, and location data.
    titles = []
    companies = []
    locations = []
    for card in soup.select("div.result-card__contents"):
        title = card.findChild("h3", recursive=False)
        company = card.findChild("h4", recursive=False)
        location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
        titles.append(title.string)
        companies.append(company.string)
        locations.append(location.string)
    
    # Inject job titles, companies, and locations into the empty dataframe
    zipped = zip(titles, companies, locations)
    for z in list(zipped):
        data=data.append({'Title' : z[0] , 'Company' : z[1], 'Location': z[2]} , ignore_index=True)
    
    # Return dataframe
    return data

In [4]:
# Example to call the function

results = scrape_linkedin_job_search('data%20analysis')
results

,Title,Company,Location
0,Willing to move back to India?,Samsung Electronics,"San Jose, CA"
1,Remote Epidemiologist,Insight Global,Denver Metropolitan Area
2,Data Analyst,Mondo,"San Francisco, CA"
3,Programming Analyst,Sony Pictures Entertainment,"Miami, FL"
4,Insights Analyst,Daymon,"Hutchinson, KS"
5,Data Scientist,Apple,"Cupertino, CA"
6,Data and Analytics Engineer,rprt,"Buffalo, NY"
7,Data Analyst - Risk and Payments,Wish,"San Francisco, CA"
8,Data Analyst/ Scientist,Claremont Consulting,"Boston, MA"
9,Data Analyst,Motion Recruitment,"Boston, MA"


## Challenge 1

The first challenge for you is to update the `scrape_linkedin_job_search` function by adding a new parameter called `num_pages`. This will allow you to search more than 25 jobs with this function. Suggested steps:

1. Go to https://www.linkedin.com/jobs/search/?keywords=data%20analysis in your browser.
1. Scroll down the left panel and click the page 2 link. Look at how the URL changes and identify the page offset parameter.
1. Add `num_pages` as a new param to the `scrape_linkedin_job_search` function. Update the function code so that it uses a "for" loop to retrieve several pages of search results.
1. Test your new function by scraping 5 pages of the search results.

Hint: Prepare for the case where there are less than 5 pages of search results. Your function should be robust enough to **not** trigger errors. Simply skip making additional searches and return all results if the search already reaches the end.

In [5]:
def scrape_linkedin_job_search(keywords, num_page):
    
    # Define the base url to be scraped.
    # All uppercase variable name signifies this is a constant and its value should never unchange
    BASE_URL = 'https://www.linkedin.com/jobs/search/?'

    # Create a request to get the data from the server 
    scrape_url = [''.join([BASE_URL, 'keywords=', keywords, '&start=', str(25*i)]) for i in range(num_page)]
    soup = []
    for url in scrape_url:
        if requests.get(url).status_code == 200:
            soup.append(BeautifulSoup(requests.get(url).text, 'html.parser'))
        else:
            break
  
    # Create an empty dataframe with the columns consisting of the information you want to capture
    columns = ['Title', 'Company', 'Location']
    data = pd.DataFrame(columns=columns)

    # Retrieve HTML code from the webpage. Parse the HTML into a list of "cards".
    # Then in each job card, extract the job title, company, and location data.
    titles = []
    companies = []
    locations = []
    for p in soup:
        for card in p.select("div.result-card__contents"):
            title = card.findChild("h3", recursive=False)
            company = card.findChild("h4", recursive=False)
            location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
            titles.append(title.string)
            companies.append(company.string)
            locations.append(location.string)
    
    # Inject job titles, companies, and locations into the empty dataframe
    zipped = zip(titles, companies, locations)
    for z in list(zipped):
        data=data.append({'Title' : z[0] , 'Company' : z[1], 'Location': z[2]} , ignore_index=True)
    
    # Return dataframe
    return data

In [4]:
scrape_linkedin_job_search('data%20analysis', 2)


,Title,Company,Location
0,"Reporting & Data Visualization, Product Analyt...",National Basketball Association (NBA),"Secaucus, NJ"
1,Willing to move back to India?,Samsung Electronics,"San Jose, CA"
2,Remote Epidemiologist,Insight Global,Denver Metropolitan Area
3,Data Analyst,Mondo,"San Francisco, CA"
4,Programming Analyst,Sony Pictures Entertainment,"Miami, FL"
5,Insights Analyst,Daymon,"Hutchinson, KS"
6,Data Scientist,Apple,"Cupertino, CA"
7,Data and Analytics Engineer,rprt,"Buffalo, NY"
8,Data Analyst - Risk and Payments,Wish,"San Francisco, CA"
9,Data Analyst,Motion Recruitment,"Boston, MA"


## Challenge 2

Further improve your function so that it can search jobs in a specific country. Add the 3rd param to your function called `country`. The steps are identical to those in Challange 1.

In [6]:
def scrape_linkedin_job_search(keywords, num_page, country):
    
    # Define the base url to be scraped.
    # All uppercase variable name signifies this is a constant and its value should never unchange
    BASE_URL = 'https://www.linkedin.com/jobs/search/?'

    # Create a request to get the data from the server 
    scrape_url = [''.join([BASE_URL, 'keywords=', keywords,'&location=', country, '&start=', str(25*i)]) for i in range(num_page)]
    soup = []
    for url in scrape_url:
        if requests.get(url).status_code == 200:
            soup.append(BeautifulSoup(requests.get(url).text, 'html.parser'))
        else:
            break

  
    # Create an empty dataframe with the columns consisting of the information you want to capture
    columns = ['Title', 'Company', 'Location']
    data = pd.DataFrame(columns=columns)

    # Retrieve HTML code from the webpage. Parse the HTML into a list of "cards".
    # Then in each job card, extract the job title, company, and location data.
    titles = []
    companies = []
    locations = []
    for p in soup:
        for card in p.select("div.result-card__contents"):
            title = card.findChild("h3", recursive=False)
            company = card.findChild("h4", recursive=False)
            location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
            titles.append(title.string)
            companies.append(company.string)
            locations.append(location.string)
    
    # Inject job titles, companies, and locations into the empty dataframe
    zipped = zip(titles, companies, locations)
    for z in list(zipped):
        data=data.append({'Title' : z[0] , 'Company' : z[1], 'Location': z[2]} , ignore_index=True)
    
    # Return dataframe
    return data

In [7]:
scrape_linkedin_job_search('data%20analysis', 2, 'Germany')


,Title,Company,Location
0,PhD Position (m/w/d) “GRACE Follow-On Data Ana...,Leibniz Universität Hannover,"Hannover, Lower Saxony, Germany"
1,(Junior) Data Scientist / Data Analyst (w/m/d),KPMG Deutschland,"Frankfurt, Hesse, Germany"
2,(Junior) Data Scientist / Data Analyst (w/m/d)...,Jobs via eFinancialCareers,"Berlin, Berlin, Germany"
3,Data Analyst: Customer Intelligence - Zalando ...,Zalando SE,"Berlin, Berlin, Germany"
4,Wekstudent (m/w/d) Data Analyst / Tableau (Dat...,Campusjäger,"Düsseldorf, North Rhine-Westphalia, Germany"
5,Data Science Analyst w/ TS,Axiologic Solutions LLC,"Vaihingen an der Enz, Baden-Württemberg, Germany"
6,Senior Data Analyst,Eurowings Digital GmbH,"Cologne, North Rhine-Westphalia, Germany"
7,Marketing Data Analyst,Pandata,"Berlin, Berlin, Germany"
8,(Junior) Data Scientist / Data Analyst (w/m/d),KPMG Deutschland,"Saarbrücken, Saarland, Germany"
9,Senior Data Analyst,Takeaway.com,"Berlin, Berlin, Germany"


## Challenge 3

Add the 4th param called `num_days` to your function to allow it to search jobs posted in the past X days. Note that in the LinkedIn job search the searched timespan is specified with the following param:

```
f_TPR=r259200
```

The number part in the param value is the number of seconds. 259,200 seconds equal to 3 days. You need to convert `num_days` to number of seconds and supply that info to LinkedIn job search.

In [8]:
def scrape_linkedin_job_search(keywords, num_page, country, num_days):
    
    # Define the base url to be scraped.
    # All uppercase variable name signifies this is a constant and its value should never unchange
    BASE_URL = 'https://www.linkedin.com/jobs/search/?'

    # Create a request to get the data from the server 
    scrape_url = [''.join([BASE_URL,'f_TRP=', str(num_days*24*60*60), 'keywords=', keywords,'&location=', country, '&start=', str(25*i)]) for i in range(num_page)]
    soup = []
    for url in scrape_url:
        if requests.get(url).status_code == 200:
            soup.append(BeautifulSoup(requests.get(url).text, 'html.parser'))
        else:
            break

  
    # Create an empty dataframe with the columns consisting of the information you want to capture
    columns = ['Title', 'Company', 'Location']
    data = pd.DataFrame(columns=columns)

    # Retrieve HTML code from the webpage. Parse the HTML into a list of "cards".
    # Then in each job card, extract the job title, company, and location data.
    titles = []
    companies = []
    locations = []
    for p in soup:
        for card in p.select("div.result-card__contents"):
            title = card.findChild("h3", recursive=False)
            company = card.findChild("h4", recursive=False)
            location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
            titles.append(title.string)
            companies.append(company.string)
            locations.append(location.string)
    
    # Inject job titles, companies, and locations into the empty dataframe
    zipped = zip(titles, companies, locations)
    for z in list(zipped):
        data=data.append({'Title' : z[0] , 'Company' : z[1], 'Location': z[2]} , ignore_index=True)
    
    # Return dataframe
    return data

In [9]:
scrape_linkedin_job_search('data%20analysis', 2, 'Germany', 1)


,Title,Company,Location
0,Head of People & Culture,Too Good To Go,"Berlin, Berlin, Germany"
1,Consultant (m/w/d),Internationaler Bund (IB),"Hamburg, Hamburg, Germany"
2,Online Coordinator (M/W/D),Estee Lauder Companies Europe,"Munich, Bavaria, Germany"
3,Investment Analyst (m/w/d),Taledo,"Berlin, Berlin, Germany"
4,Investment Manager (d/m/w),NÜRNBERGER Versicherungsgruppe,"Nuremberg, Bavaria, Germany"
5,(Junior) Consultant (m/f/d) Consumer Panel Travel,GfK,"Nuremberg, Bavaria, Germany"
6,Data Scientist (m/f/d),Porsche Consulting Brasil,"Hamburg, Hamburg, Germany"
7,Learning Specialist (m/w/d),TU Dortmund University,"Dortmund, North Rhine-Westphalia, Germany"
8,Brand Marketing Manager (m/w/d),Bertelsmann SE & Co. KGaA,"Bochum, North Rhine-Westphalia, Germany"
9,Project Office Assistant (m/w/d),Schreiner,"Hochdorf, Baden-Württemberg, Germany"


## Bonus Challenge

Allow your function to also retrieve the "Seniority Level" of each job searched. Note that the Seniority Level info is not in the initial search results. You need to make a separate search request for each job card based on the `currentJobId` value which you can extract from the job card HTML.

After you obtain the Seniority Level info, update the function and add it to a new column of the returned dataframe.